In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configurações para os gráficos
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 7)

print("Bibliotecas importadas.")

Bibliotecas importadas.


In [2]:
# Carregar a tabela de features final, resultado de todo o nosso trabalho de EDA
path_final_table = '../data/processed/feature_table_final.parquet'
df = pd.read_parquet(path_final_table)

print("Tabela de features final carregada.")
print(f"A tabela contém {df.shape[0]} dias e {df.shape[1]} colunas.")
display(df.head())

Tabela de features final carregada.
A tabela contém 2787 dias e 13 colunas.


,deficit_diario_mwh,nivel_risco,programada,verificada,diferenca_verif_prog,geracao_total_diaria_go,geracao_fotovoltaica_diaria,geracao_hidroelétrica_diaria,geracao_térmica_diaria,total_mwh_restrito_go,saldo_intercambio_seco,ear_percentual_seco,ena_percentual_mlt_seco
timestamp,,,,,,,,,,,,,
2018-01-25,732.333333,alto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.231899,88.28579712
2018-01-26,0.000000,baixo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.335600,86.38259888
2018-01-27,0.000000,baixo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.594200,84.68139648
2018-01-28,64.901667,medio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.782801,83.22190094
2018-01-29,10.850000,baixo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.804199,82.21679688


In [3]:
# --- Célula 3: Engenharia de Features - Janelas Deslizantes (Rolling Windows & Lag) ---

# --- Features de Carga (Demanda) ---
# Média da carga verificada nos últimos 7 dias
df['carga_media_7d'] = df['verificada'].rolling(window=7).mean()
# Desvio padrão da carga nos últimos 7 dias (mede a volatilidade)
df['carga_std_7d'] = df['verificada'].rolling(window=7).std()

# --- Features de Geração (Oferta) ---
# Média da geração total em Goiás nos últimos 7 dias
df['geracao_media_7d'] = df['geracao_total_diaria_go'].rolling(window=7).mean()

# --- Features Hídricas (Segurança do Sistema) ---
# Valor do EAR do dia anterior (lag de 1 dia)
df['ear_ontem'] = df['ear_percentual_seco'].shift(1)
# Diferença (queda ou subida) do nível do reservatório nos últimos 3 dias
df['ear_diff_3d'] = df['ear_percentual_seco'].diff(periods=3)

# --- NOVA ADIÇÃO: Features de Precipitação Acumulada ---
# Esta feature só existirá se o notebook 08 (meteorologia) já tiver sido executado
if 'precipitacao_mm' in df.columns:
    # Soma da precipitação nos últimos 14 dias
    df['precip_acumulada_14d'] = df['precipitacao_mm'].rolling(window=14).sum()
    # Soma da precipitação nos últimos 30 dias
    df['precip_acumulada_30d'] = df['precipitacao_mm'].rolling(window=30).sum()
    print("Features de precipitação acumulada criadas com sucesso.")
else:
    print("AVISO: Coluna 'precipitacao_mm' não encontrada. Pule a criação das features de precipitação.")

print("\nFeatures de janela deslizante e lag criadas com sucesso.")

AVISO: Coluna 'precipitacao_mm' não encontrada. Pule a criação das features de precipitação.

Features de janela deslizante e lag criadas com sucesso.


In [4]:
# Feature que representa a "folga" ou "aperto" entre oferta e demanda no dia
df['margem_oferta_demanda'] = df['geracao_total_diaria_go'] - df['verificada']

# Feature que mede a pressão da demanda sobre o nível de segurança dos reservatórios
# Quão alta é a carga em relação ao nível do EAR?
# Adicionamos 0.01 para evitar divisão por zero caso o EAR seja 0
df['pressao_demanda_ear'] = df['verificada'] / (df['ear_percentual_seco'] + 0.01)

print("Features de interação criadas com sucesso.")

Features de interação criadas com sucesso.


In [5]:
print(f"Shape do dataframe ANTES de remover NaNs: {df.shape}")

# Inspeciona quantos NaNs foram criados em cada nova coluna
print("\nContagem de NaNs criados:")
print(df[['carga_media_7d', 'ear_ontem', 'ear_diff_3d']].isnull().sum())

# Remove todas as linhas que contenham qualquer valor NaN
df.dropna(inplace=True)

print(f"\nShape do dataframe DEPOIS de remover NaNs: {df.shape}")

Shape do dataframe ANTES de remover NaNs: (2787, 20)

Contagem de NaNs criados:
carga_media_7d    6
ear_ontem         1
ear_diff_3d       3
dtype: int64

Shape do dataframe DEPOIS de remover NaNs: (2781, 20)


In [6]:
# --- Célula 6: Salvando a Tabela Final para Modelagem ---

# Define e cria o diretório de saída se não existir
output_path = '../data/processed/modeling_table.parquet'
output_dir = os.path.dirname(output_path)
os.makedirs(output_dir, exist_ok=True)

# CORREÇÃO: Certifique-se de que o nome do DataFrame aqui ('df') é o mesmo
# que você usou nas células 2, 3, 4 e 5.
df.to_parquet(output_path)

print(f"\nTABELA FINAL PARA MODELAGEM salva com sucesso em: {output_path}")
print(f"A tabela contém {df.shape[0]} dias e {df.shape[1]} colunas.")


TABELA FINAL PARA MODELAGEM salva com sucesso em: ../data/processed/modeling_table.parquet
A tabela contém 2781 dias e 20 colunas.
